[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb)

In [1]:
!wget https://github.com/LamUong/FacialExpressionRecognition/blob/master/badtrainingdata.txt

--2018-06-19 05:32:15--  https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/download/fer2013.tar.gz
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz [following]
--2018-06-19 05:32:15--  https://www.kaggle.com/account/login?ReturnUrl=%2fc%2fchallenges-in-representation-learning-facial-expression-recognition-challenge%2fdownload%2ffer2013.tar.gz
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 8229 (8.0K) [text/html]
Saving to: ‘fer2013.tar.gz’

fer2013.tar.gz      100%[===================>]   8.04K  --.-KB/s    in 0s      

2018-06-19 05:32:15 (149 MB/s) - ‘fer2013.tar.gz’ saved [8229/8229]



In [1]:
from google.colab import files
uploaded = files.upload()

Saving fer2013.tar to fer2013.tar


In [3]:
!ls

datalab  fer2013.tar  fer2013.tar.gz


In [42]:
import os
!export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ.pop('TF_ENABLE_WINOGRAD_NONFUSED', None)

'1'

In [2]:
!tar xvf fer2013.tar
!ls

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/
datalab  fer2013  fer2013.tar


In [3]:

import csv
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, InputLayer
from keras.layers import Convolution2D, SeparableConv2D, MaxPooling2D, BatchNormalization 
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
img_rows, img_cols = 48, 48
batch_size = 128
classes = 7
epoch = 100
img_channels = 1

In [0]:
import csv
f = open('fer2013/fer2013.csv')
csv_f = csv.reader(f)


In [0]:
train_x = []
train_y = []
val_x =[]
val_y =[]

In [0]:
num=0
for row in csv_f:
  #print(row)
  num = num +1
  #print(num)
  temp_list = []
  for pixel in row[1].split( ):
    temp_list.append(int(pixel))

  if str(row[2]) == "Training":
    train_y.append(int(row[0]))
    train_x.append(temp_list) 
  elif str(row[2]) == "PublicTest":
    val_y.append(int(row[0]))
    val_x.append(temp_list)

In [0]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

In [0]:
train_x = train_x.reshape(train_x.shape[0], 48, 48)
train_x = train_x.reshape(train_x.shape[0], 48, 48, 1 )
train_y = np_utils.to_categorical(train_y, 7)

In [0]:
val_x = val_x.reshape(val_x.shape[0], 48, 48)
val_x = val_x.reshape(val_x.shape[0], 48, 48, 1)
val_y = np_utils.to_categorical(val_y, 7)

In [0]:
#print(train_x[1].shape)
train_x_pil = train_x.reshape(train_x.shape[0], 48, 48)
from PIL import Image
from numpy import array
img = Image.fromarray(train_x[1])
img.save("output.png")


In [0]:
train_x = train_x.astype('float32')
train_x = train_x / 255.0
val_x = val_x.astype('float32')
val_x = val_x / 255.0


In [0]:
input_shape = (img_rows, img_cols, img_channels)
model = Sequential()
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same',
                            name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

#model.add(Flatten())
'''
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(.5))

model.add(Dense(2048))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(.5))
'''
model.add(InputLayer(input_shape=(3, 3, 1024)))
model.add(GlobalAveragePooling2D())
model.add(Dense(7))
model.add(Activation('softmax'))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (SeparableConv2D (None, 48, 48, 64)        137       
_________________________________________________________________
batch_normalization_27 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 48, 48, 64)        0         
_________________________________________________________________
separable_conv2d_22 (Separab (None, 48, 48, 64)        4736      
_________________________________________________________________
batch_normalization_28 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 24, 24, 64)        0         
__________

In [0]:
model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
filepath='Model.{epoch:02d}.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=Trues, mode='auto')


In [0]:
model.load_weights('Model.50.hdf5')

In [56]:
# LReLU
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=50, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3661 - acc: 0.8669 - val_loss: 1.4124 - val_acc: 0.6110

Epoch 00001: saving model to Model.01.hdf5
Epoch 2/50
26496/28709 [==========================>...] - ETA: 3s - loss: 0.3551 - acc: 0.8712

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3560 - acc: 0.8701 - val_loss: 1.5030 - val_acc: 0.6113

Epoch 00002: saving model to Model.02.hdf5
Epoch 3/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3557 - acc: 0.8704 - val_loss: 1.3140 - val_acc: 0.6400

Epoch 00003: saving model to Model.03.hdf5
Epoch 4/50
10880/28709 [==========>...................] - ETA: 31s - loss: 0.3338 - acc: 0.8781

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3471 - acc: 0.8747 - val_loss: 1.3368 - val_acc: 0.6395

Epoch 00004: saving model to Model.04.hdf5
Epoch 5/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3384 - acc: 0.8795 - val_loss: 1.4704 - val_acc: 0.6216

Epoch 00005: saving model to Model.05.hdf5
Epoch 6/50
 7936/28709 [=======>......................] - ETA: 37s - loss: 0.3256 - acc: 0.8851

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3426 - acc: 0.8764 - val_loss: 1.4135 - val_acc: 0.6342

Epoch 00006: saving model to Model.06.hdf5
Epoch 7/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3301 - acc: 0.8799 - val_loss: 1.3993 - val_acc: 0.6208

Epoch 00007: saving model to Model.07.hdf5
Epoch 8/50
 7424/28709 [======>.......................] - ETA: 37s - loss: 0.3113 - acc: 0.8848

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3218 - acc: 0.8829 - val_loss: 1.3684 - val_acc: 0.6066

Epoch 00008: saving model to Model.08.hdf5
Epoch 9/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3173 - acc: 0.8843 - val_loss: 1.4625 - val_acc: 0.6247

Epoch 00009: saving model to Model.09.hdf5
Epoch 10/50
 7296/28709 [======>.......................] - ETA: 38s - loss: 0.2883 - acc: 0.8957

...

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2188 - acc: 0.9224 - val_loss: 1.6097 - val_acc: 0.6211

Epoch 00042: saving model to Model.42.hdf5
Epoch 43/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2095 - acc: 0.9257 - val_loss: 1.6839 - val_acc: 0.6096

Epoch 00043: saving model to Model.43.hdf5
Epoch 44/50
 7168/28709 [======>.......................] - ETA: 38s - loss: 0.2057 - acc: 0.9268

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2166 - acc: 0.9233 - val_loss: 1.9675 - val_acc: 0.5821

Epoch 00044: saving model to Model.44.hdf5
Epoch 45/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2054 - acc: 0.9262 - val_loss: 1.4574 - val_acc: 0.6356

Epoch 00045: saving model to Model.45.hdf5
Epoch 46/50
 7168/28709 [======>.......................] - ETA: 38s - loss: 0.1958 - acc: 0.9291

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2068 - acc: 0.9262 - val_loss: 1.6456 - val_acc: 0.6158

Epoch 00046: saving model to Model.46.hdf5
Epoch 47/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2158 - acc: 0.9252 - val_loss: 1.6853 - val_acc: 0.6211

Epoch 00047: saving model to Model.47.hdf5
Epoch 48/50
 7168/28709 [======>.......................] - ETA: 38s - loss: 0.1829 - acc: 0.9361

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2027 - acc: 0.9292 - val_loss: 1.6892 - val_acc: 0.6233

Epoch 00048: saving model to Model.48.hdf5
Epoch 49/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2053 - acc: 0.9264 - val_loss: 1.5443 - val_acc: 0.6344

Epoch 00049: saving model to Model.49.hdf5
Epoch 50/50
 7168/28709 [======>.......................] - ETA: 38s - loss: 0.2020 - acc: 0.9270

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2052 - acc: 0.9279 - val_loss: 1.5657 - val_acc: 0.6470

Epoch 00050: saving model to Model.50.hdf5
--- 2669.086063146591 seconds ---


In [50]:
# input normalized
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 162s 6ms/step - loss: 2.0189 - acc: 0.2407 - val_loss: 1.8390 - val_acc: 0.2856
Epoch 2/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.8329 - acc: 0.2909 - val_loss: 1.7546 - val_acc: 0.3391
Epoch 3/10
  896/28709 [..............................] - ETA: 2:27 - loss: 1.7662 - acc: 0.3058

28709/28709 [==============================] - 157s 5ms/step - loss: 1.7695 - acc: 0.3230 - val_loss: 1.8986 - val_acc: 0.3093
Epoch 4/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.7122 - acc: 0.3484 - val_loss: 1.6657 - val_acc: 0.3611
Epoch 5/10
10880/28709 [==========>...................] - ETA: 1:34 - loss: 1.6901 - acc: 0.3509

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6773 - acc: 0.3604 - val_loss: 1.7014 - val_acc: 0.3494
Epoch 6/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.6553 - acc: 0.3702 - val_loss: 1.6194 - val_acc: 0.3859
Epoch 7/10
13312/28709 [============>.................] - ETA: 1:20 - loss: 1.6141 - acc: 0.3857

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6129 - acc: 0.3857 - val_loss: 1.6001 - val_acc: 0.3923
Epoch 8/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.5920 - acc: 0.3924 - val_loss: 1.6594 - val_acc: 0.4035
Epoch 9/10
13824/28709 [=============>................] - ETA: 1:18 - loss: 1.5641 - acc: 0.4065

28709/28709 [==============================] - 157s 5ms/step - loss: 1.5559 - acc: 0.4044 - val_loss: 1.5507 - val_acc: 0.4113
Epoch 10/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.5341 - acc: 0.4100 - val_loss: 1.5732 - val_acc: 0.4068
--- 1574.3259291648865 seconds ---


In [0]:

import time
start_time = time.time()

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(train_x)

model.fit_generator(datagen.flow(train_x, train_y,
                    batch_size=batch_size),
                    steps_per_epoch=(train_x.shape[0]/batch_size),
                    epochs=10,
                    validation_data=(val_x, val_y),
                    callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))